# Cost Analysis

## 1. Import Library

In [1]:
import pandas as pd
import numpy as np
import pyodbc # SQL Connection
import sqlCredentials as sql

## 2. Connect to Databases

In [2]:
#PROCUREMENTDB

proc_db = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'Server=52.86.56.66;'
    'Database=PROCUREMENTDB;'
    'UID='+sql.username+';'
    'PWD='+sql.password+';'
    'Trusted_connection=no;'
)

#BookXCenterProduction
prod_db = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'Server=52.86.56.66;'
    'Database=BookXCenterProduction;'
    'UID='+sql.username+';'
    'PWD='+sql.password+';'
    'Trusted_connection=no;'
)

## 3. Define Clean up tool

In [3]:
def clean_up(phrase):
    phrase = phrase.lstrip()
    phrase = phrase.rstrip()
    phrase = phrase.upper()
    return phrase

## 4. Define Tables

In [4]:
# Dictionary
pub_dict = """
SELECT *
FROM dbo.PublisherDictionary
"""
# LP All currencies
all_lp = """
SELECT
	[Isbn]
	,[Currency]
	,[Price]
FROM
	[Isbn].[ListPrice]
"""
# Bibliography
bilblo_sql = """
SELECT Isbn
    , Title
    , Publisher
    , Author
FROM 
    Isbn.Bibliography
"""
# Echange Rate
exchange_rate = """
SELECT 
    name,
    rate
FROM dbo.XChange
"""
#Supplier Mega List
megalist_sql = """
SELECT 
    [ISBN]
    ,[Supplier]
    ,[Publisher]
    ,[Currency]
    ,[ListPrice]
    ,[Discount]
    ,[CostUnitPrice]
    ,[MaxQtyPerOrder]
    ,[ShipmentOrigin]
    ,[UnitShippingCost]
    ,[SeaFreightCost] AS sea_shipping_cost
  FROM 
    [Process].[SupplierMegaList]
"""

#Import supplier Excel
#Import short_discount
#Import restrictions

### 4.1 Import Dictionary and clean it

In [5]:
pub_dict = pd.read_sql(pub_dict, proc_db)
pub_dict.head()

,PublisherLong,PublisherShort
0,Glencoe McGraw Hill,MCGRAW
1,Glencoe McGraw-Hill,MCGRAW
2,McGraw Create,MCGRAW
3,MCGRAW HE,MCGRAW HE
4,MCGRAW HE - ISE,MCGRAW HE - ISE


In [6]:
pub_dict.columns = map(str.lower, pub_dict.columns)
pub_dict = pub_dict.fillna('N/A')
for col in list(pub_dict.columns):
    pub_dict[col] = pub_dict.apply(lambda x: clean_up(x[col]), axis =1)

pub_dict.head()

,publisherlong,publishershort
0,GLENCOE MCGRAW HILL,MCGRAW
1,GLENCOE MCGRAW-HILL,MCGRAW
2,MCGRAW CREATE,MCGRAW
3,MCGRAW HE,MCGRAW HE
4,MCGRAW HE - ISE,MCGRAW HE - ISE


### 4.2 Import Bibliography and clean it

In [7]:
# IMport and read biblio
biblio = pd.read_sql(bilblo_sql, prod_db)
biblio.head()

,Isbn,Title,Publisher,Author
0,9780000000057,"Text, Music, Context: A Resource Item #: S05...",CPH,LCMS
1,9780000001030,Surgical Procedures (2016),CIMC,MAVCC
2,9780000001061,Anatomy & Physiology Student Edition,CIMC,CIMC
3,9780000001146,Membean Student License,Archer,None
4,9780000002402,None,LWW,None


In [8]:
biblio.columns = map(str.lower, biblio.columns)
biblio = biblio.fillna('N/A')
biblio['isbn'] = biblio['isbn'].astype(str)
for col in list(biblio.columns):
    biblio[col] = biblio.apply(lambda x: clean_up(x[col]), axis =1)
biblio.head()

,isbn,title,publisher,author
0,9780000000057,"TEXT, MUSIC, CONTEXT: A RESOURCE ITEM #: S05...",CPH,LCMS
1,9780000001030,SURGICAL PROCEDURES (2016),CIMC,MAVCC
2,9780000001061,ANATOMY & PHYSIOLOGY STUDENT EDITION,CIMC,CIMC
3,9780000001146,MEMBEAN STUDENT LICENSE,ARCHER,N/A
4,9780000002402,N/A,LWW,N/A


Now we Merge bibliography with the dictionary

In [9]:
biblio2 = pd.merge(biblio, pub_dict, how='left', left_on = 'publisher', right_on = 'publisherlong' )
biblio2.drop(columns = ['publisherlong'], inplace = True)
biblio2.rename(columns={'publishershort': 'pub'}, inplace = True)
biblio2.head()

,isbn,title,publisher,author,pub
0,9780000000057,"TEXT, MUSIC, CONTEXT: A RESOURCE ITEM #: S05...",CPH,LCMS,NaN
1,9780000001030,SURGICAL PROCEDURES (2016),CIMC,MAVCC,NaN
2,9780000001061,ANATOMY & PHYSIOLOGY STUDENT EDITION,CIMC,CIMC,NaN
3,9780000001146,MEMBEAN STUDENT LICENSE,ARCHER,N/A,NaN
4,9780000002402,N/A,LWW,N/A,LWW


### 4.3 Import SupplierMegaList and clean it

In [10]:
megalist = pd.read_sql(megalist_sql,proc_db)
megalist.head()

,ISBN,Supplier,Publisher,Currency,ListPrice,Discount,CostUnitPrice,MaxQtyPerOrder,ShipmentOrigin,UnitShippingCost,sea_shipping_cost
0,9780262530873,BILLSON,UNIVERSITY PRESS,GBP,4.99,0.38,3.09,1000,UK,3.47,1.0
1,9780262530989,BILLSON,UNIVERSITY PRESS,GBP,25.00,0.38,15.50,1000,UK,3.47,1.0
2,9781861345806,Zookal,THAMES & HUDSON,AUD,0.00,0.00,28.00,150,AUS,3.47,1.0
3,9780262531092,BILLSON,UNIVERSITY PRESS,GBP,37.95,0.38,23.53,1000,UK,3.47,1.0
4,9781861345837,Zookal,THAMES & HUDSON,AUD,0.00,0.00,35.00,150,AUS,3.47,1.0


In [11]:
megalist.columns = map(str.lower, megalist.columns)
megalist = megalist.fillna('N/A')
megalist['isbn'] = megalist['isbn'].astype(str)
for col in ['isbn', 'supplier', 'publisher', 'currency', 'shipmentorigin']:
    megalist[col] = megalist.apply(lambda x: clean_up(x[col]), axis =1)
megalist.head()

,isbn,supplier,publisher,currency,listprice,discount,costunitprice,maxqtyperorder,shipmentorigin,unitshippingcost,sea_shipping_cost
0,9780262530873,BILLSON,UNIVERSITY PRESS,GBP,4.99,0.38,3.09,1000,UK,3.47,1.0
1,9780262530989,BILLSON,UNIVERSITY PRESS,GBP,25.00,0.38,15.50,1000,UK,3.47,1.0
2,9781861345806,ZOOKAL,THAMES & HUDSON,AUD,0.00,0.00,28.00,150,AUS,3.47,1.0
3,9780262531092,BILLSON,UNIVERSITY PRESS,GBP,37.95,0.38,23.53,1000,UK,3.47,1.0
4,9781861345837,ZOOKAL,THAMES & HUDSON,AUD,0.00,0.00,35.00,150,AUS,3.47,1.0


In [12]:
megalist_pub = pd.merge(megalist, biblio2, how='left', on = 'isbn' )
megalist_pub.head()

,isbn,supplier,publisher_x,currency,listprice,discount,costunitprice,maxqtyperorder,shipmentorigin,unitshippingcost,sea_shipping_cost,title,publisher_y,author,pub
0,9780262530873,BILLSON,UNIVERSITY PRESS,GBP,4.99,0.38,3.09,1000,UK,3.47,1.0,N/A,UNIVERSITY PRESS,N/A,UNIVERSITY PRESS
1,9780262530989,BILLSON,UNIVERSITY PRESS,GBP,25.00,0.38,15.50,1000,UK,3.47,1.0,N/A,UNIVERSITY PRESS,N/A,UNIVERSITY PRESS
2,9781861345806,ZOOKAL,THAMES & HUDSON,AUD,0.00,0.00,28.00,150,AUS,3.47,1.0,N/A,N/A,N/A,N/A
3,9780262531092,BILLSON,UNIVERSITY PRESS,GBP,37.95,0.38,23.53,1000,UK,3.47,1.0,N/A,UNIVERSITY PRESS,N/A,UNIVERSITY PRESS
4,9781861345837,ZOOKAL,THAMES & HUDSON,AUD,0.00,0.00,35.00,150,AUS,3.47,1.0,N/A,N/A,N/A,N/A


In [13]:
#cleaning pub name
megalist_pub['pub'] = np.where(
    megalist_pub['pub'] == 'N/A'
    , megalist_pub['publisher_x']
    , megalist_pub['pub'])
    
# centralizing price in one column
megalist_pub['price'] = np.where(megalist_pub['listprice'] == 0.00, megalist_pub['costunitprice'], megalist_pub['listprice'])

#clean column names
megalist_pub.rename(columns={'unitshippingcost': 'shipping_cost', 'maxqtyperorder': 'max_qty', 'shipmentorigin': 'origin'}, inplace = True)

#Arrange columns and drop unnecessary columns

megalist_pub = megalist_pub[['isbn', 'title', 'pub', 'author', 'supplier', 'currency', 'price', 'discount', 'shipping_cost','sea_shipping_cost', 'max_qty', 'origin']]

megalist_pub.head()

,isbn,title,pub,author,supplier,currency,price,discount,shipping_cost,sea_shipping_cost,max_qty,origin
0,9780262530873,N/A,UNIVERSITY PRESS,N/A,BILLSON,GBP,4.99,0.38,3.47,1.0,1000,UK
1,9780262530989,N/A,UNIVERSITY PRESS,N/A,BILLSON,GBP,25.00,0.38,3.47,1.0,1000,UK
2,9781861345806,N/A,THAMES & HUDSON,N/A,ZOOKAL,AUD,28.00,0.00,3.47,1.0,150,AUS
3,9780262531092,N/A,UNIVERSITY PRESS,N/A,BILLSON,GBP,37.95,0.38,3.47,1.0,1000,UK
4,9781861345837,N/A,THAMES & HUDSON,N/A,ZOOKAL,AUD,35.00,0.00,3.47,1.0,150,AUS


### 4.3 Import all_lp and clean it

In [14]:
all_lp = pd.read_sql(all_lp, prod_db)
all_lp.head()

,Isbn,Currency,Price
0,9780030699160,USD,32.40
1,9780030741524,USD,18.30
2,9780030751196,GBP,54.99
3,9780030769672,USD,383.20
4,9780030779916,USD,31.80


In [15]:
all_lp.columns = map(str.lower, all_lp.columns)
all_lp = all_lp.fillna('N/A')
all_lp['isbn'] = all_lp['isbn'].astype(str)
for col in ['isbn','currency']:
    all_lp[col] = all_lp.apply(lambda x: clean_up(x[col]), axis =1)

all_lp.head()

,isbn,currency,price
0,9780030699160,USD,32.40
1,9780030741524,USD,18.30
2,9780030751196,GBP,54.99
3,9780030769672,USD,383.20
4,9780030779916,USD,31.80


#### 4.3.1 Get biblio with LP

In [16]:
biblio_lp = pd.merge(all_lp, biblio2, how='left', on = 'isbn' )
biblio_lp.drop(columns = ['pub'], inplace = True)
biblio_lp.head()

,isbn,currency,price,title,publisher,author
0,9780030699160,USD,32.40,N/A,HOUGHTON MIFFLIN,N/A
1,9780030741524,USD,18.30,NUEVAS VISTAS: CUADERNO PRACTICA INTRO,HOLT MCDOUGAL,HOLT RINEHART AND WINSTON
2,9780030751196,GBP,54.99,N/A,CENGAGE,N/A
3,9780030769672,USD,383.20,N/A,HOUGHTON MIFFLIN,N/A
4,9780030779916,USD,31.80,N/A,HOUGHTON MIFFLIN,N/A


### 4.4 Import Sup_by_pub and clean it

In [17]:
sup_pub = pd.read_excel('../../../../Vendors/.  Vendors Details\supplier_procurement_details.xlsx',  sheet_name='pub')
sup_pub.head()

,Supplier,Publisher,Currency,Discount,Shipping Cost,max quantity,Min,ShipmentOrigin,Shipping Cost Sea
0,COINFO,3D TOTAL,AUD,0.22,3.3,10000,0,AUS,1.0
1,COINFO,ABC CLIO,AUD,0.30,3.3,10000,0,AUS,1.0
2,COINFO,AIAA,AUD,0.30,3.3,10000,0,AUS,1.0
3,COINFO,ALIEN & UNWIN,AUD,0.30,3.3,10000,0,AUS,1.0
4,COINFO,Amer Psych Ass.,AUD,0.33,3.3,10000,0,AUS,1.0


In [18]:
sup_pub.columns = map(str.lower, sup_pub.columns)
sup_pub = sup_pub.fillna('N/A')
for col in ['supplier','publisher', 'currency', 'shipmentorigin']:
    sup_pub[col] = sup_pub.apply(lambda x: clean_up(x[col]), axis =1)
sup_pub.head()

,supplier,publisher,currency,discount,shipping cost,max quantity,min,shipmentorigin,shipping cost sea
0,COINFO,3D TOTAL,AUD,0.22,3.3,10000,0,AUS,1.0
1,COINFO,ABC CLIO,AUD,0.30,3.3,10000,0,AUS,1.0
2,COINFO,AIAA,AUD,0.30,3.3,10000,0,AUS,1.0
3,COINFO,ALIEN & UNWIN,AUD,0.30,3.3,10000,0,AUS,1.0
4,COINFO,AMER PSYCH ASS.,AUD,0.33,3.3,10000,0,AUS,1.0


#### 4.4.1 Merge Supplier by Publisher with biblio

In [19]:
sup_pub_biblio = pd.merge(biblio_lp, sup_pub, how='inner', left_on = ['currency', 'publisher'], right_on = ['currency', 'publisher'])
sup_pub_biblio.head()

,isbn,currency,price,title,publisher,author,supplier,discount,shipping cost,max quantity,min,shipmentorigin,shipping cost sea
0,9780030751196,GBP,54.99,N/A,CENGAGE,N/A,ABE,0.41,3.47,10000,0,UK,1.0
1,9780030723988,GBP,71.99,N/A,CENGAGE,N/A,ABE,0.41,3.47,10000,0,UK,1.0
2,9780030758997,GBP,74.99,FRENCH FOR ORAL AND WRITTEN REVIEW,CENGAGE,CHARLES CARLUT; WALTER MEIDEN,ABE,0.41,3.47,10000,0,UK,1.0
3,9780030973697,GBP,109.00,PRINCIPLES OF BIOCHEMISTRY WITH A HUMAN FOCUS,CENGAGE,REGINALD H. GARRETT; CHARLES M. GRISHAM,ABE,0.41,3.47,10000,0,UK,1.0
4,9780008179038,GBP,6.99,AQA A-LEVEL PHYSICS PRACTICE TEST PAPERS,CENGAGE,N/A,ABE,0.41,3.47,10000,0,UK,1.0


In [20]:

sup_pub_biblio.rename(columns={'publisher': 'pub','shipping cost': 'shipping_cost', 'max quantity': 'max_qty', 'shipmentorigin': 'origin', 'shipping cost sea' : 'sea_shipping_cost'}, inplace = True)

sup_pub_biblio.head()

,isbn,currency,price,title,pub,author,supplier,discount,shipping_cost,max_qty,min,origin,sea_shipping_cost
0,9780030751196,GBP,54.99,N/A,CENGAGE,N/A,ABE,0.41,3.47,10000,0,UK,1.0
1,9780030723988,GBP,71.99,N/A,CENGAGE,N/A,ABE,0.41,3.47,10000,0,UK,1.0
2,9780030758997,GBP,74.99,FRENCH FOR ORAL AND WRITTEN REVIEW,CENGAGE,CHARLES CARLUT; WALTER MEIDEN,ABE,0.41,3.47,10000,0,UK,1.0
3,9780030973697,GBP,109.00,PRINCIPLES OF BIOCHEMISTRY WITH A HUMAN FOCUS,CENGAGE,REGINALD H. GARRETT; CHARLES M. GRISHAM,ABE,0.41,3.47,10000,0,UK,1.0
4,9780008179038,GBP,6.99,AQA A-LEVEL PHYSICS PRACTICE TEST PAPERS,CENGAGE,N/A,ABE,0.41,3.47,10000,0,UK,1.0


#### 4.5 Import short_disc and clean it

In [21]:
sp_disc = pd.read_excel("../../../../Vendors/. Short Discounted/all_short_discount_list.xlsx")
sp_disc.head()

,SUPPLIER,ISBN,DISCOUNT
0,ABE,9781408041734,0.2
1,ABE,9781408041918,0.2
2,ABE,9781408056509,0.2
3,ABE,9781408060377,0.2
4,ABE,9780534944827,0.3


In [22]:
sp_disc.columns = map(str.lower, sp_disc.columns)
sp_disc = sp_disc.fillna('N/A')
sp_disc['isbn'] = sp_disc['isbn'].astype(str)
for col in ['supplier', 'isbn']:
    sp_disc[col] = sp_disc.apply(lambda x: clean_up(x[col]), axis =1)
sp_disc.head()

,supplier,isbn,discount
0,ABE,9781408041734,0.2
1,ABE,9781408041918,0.2
2,ABE,9781408056509,0.2
3,ABE,9781408060377,0.2
4,ABE,9780534944827,0.3


#### 4.5.1 Merge sup_pub_biblio by sp_disc with biblio

In [23]:
sup_biblio_disc = pd.merge(sup_pub_biblio, sp_disc, how='left', on = ['supplier', 'isbn'])
sup_biblio_disc.head()

,isbn,currency,price,title,pub,author,supplier,discount_x,shipping_cost,max_qty,min,origin,sea_shipping_cost,discount_y
0,9780030751196,GBP,54.99,N/A,CENGAGE,N/A,ABE,0.41,3.47,10000,0,UK,1.0,NaN
1,9780030723988,GBP,71.99,N/A,CENGAGE,N/A,ABE,0.41,3.47,10000,0,UK,1.0,NaN
2,9780030758997,GBP,74.99,FRENCH FOR ORAL AND WRITTEN REVIEW,CENGAGE,CHARLES CARLUT; WALTER MEIDEN,ABE,0.41,3.47,10000,0,UK,1.0,NaN
3,9780030973697,GBP,109.00,PRINCIPLES OF BIOCHEMISTRY WITH A HUMAN FOCUS,CENGAGE,REGINALD H. GARRETT; CHARLES M. GRISHAM,ABE,0.41,3.47,10000,0,UK,1.0,NaN
4,9780008179038,GBP,6.99,AQA A-LEVEL PHYSICS PRACTICE TEST PAPERS,CENGAGE,N/A,ABE,0.41,3.47,10000,0,UK,1.0,NaN


In [24]:
sup_biblio_disc['discount'] = np.where(sup_biblio_disc['discount_y'].isna(), sup_biblio_disc['discount_x'],sup_biblio_disc['discount_y'])


#Drop unneccessary Columns

sup_biblio_disc.head()

,isbn,currency,price,title,pub,author,supplier,discount_x,shipping_cost,max_qty,min,origin,sea_shipping_cost,discount_y,discount
0,9780030751196,GBP,54.99,N/A,CENGAGE,N/A,ABE,0.41,3.47,10000,0,UK,1.0,NaN,0.41
1,9780030723988,GBP,71.99,N/A,CENGAGE,N/A,ABE,0.41,3.47,10000,0,UK,1.0,NaN,0.41
2,9780030758997,GBP,74.99,FRENCH FOR ORAL AND WRITTEN REVIEW,CENGAGE,CHARLES CARLUT; WALTER MEIDEN,ABE,0.41,3.47,10000,0,UK,1.0,NaN,0.41
3,9780030973697,GBP,109.00,PRINCIPLES OF BIOCHEMISTRY WITH A HUMAN FOCUS,CENGAGE,REGINALD H. GARRETT; CHARLES M. GRISHAM,ABE,0.41,3.47,10000,0,UK,1.0,NaN,0.41
4,9780008179038,GBP,6.99,AQA A-LEVEL PHYSICS PRACTICE TEST PAPERS,CENGAGE,N/A,ABE,0.41,3.47,10000,0,UK,1.0,NaN,0.41


In [25]:
sup_biblio_disc = sup_biblio_disc[['isbn', 'title', 'pub', 'author', 'supplier', 'currency', 'price', 'discount', 'shipping_cost', 'sea_shipping_cost', 'max_qty', 'origin']]
sup_biblio_disc.head()

,isbn,title,pub,author,supplier,currency,price,discount,shipping_cost,sea_shipping_cost,max_qty,origin
0,9780030751196,N/A,CENGAGE,N/A,ABE,GBP,54.99,0.41,3.47,1.0,10000,UK
1,9780030723988,N/A,CENGAGE,N/A,ABE,GBP,71.99,0.41,3.47,1.0,10000,UK
2,9780030758997,FRENCH FOR ORAL AND WRITTEN REVIEW,CENGAGE,CHARLES CARLUT; WALTER MEIDEN,ABE,GBP,74.99,0.41,3.47,1.0,10000,UK
3,9780030973697,PRINCIPLES OF BIOCHEMISTRY WITH A HUMAN FOCUS,CENGAGE,REGINALD H. GARRETT; CHARLES M. GRISHAM,ABE,GBP,109.00,0.41,3.47,1.0,10000,UK
4,9780008179038,AQA A-LEVEL PHYSICS PRACTICE TEST PAPERS,CENGAGE,N/A,ABE,GBP,6.99,0.41,3.47,1.0,10000,UK


### 4.6 Create table for WC

In [26]:
can_sup = biblio_lp[biblio_lp['currency'] == 'CND']
can_sup.columns = map(str.lower, can_sup.columns)
can_sup['supplier'] = 'WC'
can_sup['discount'] = np.where(can_sup['price']<150, 0.1, np.where((can_sup['price'] >=150) & (can_sup['price']<200),0.12,0.14))
can_sup['max_qty'] = 10000
can_sup['origin'] = 'CAN'
can_sup['shipping_cost'] = 2
can_sup.rename(columns={'publisher': 'pub'}, inplace = True)
can_sup['currency'] = 'CAD'
can_sup['sea_shipping_cost'] = 2

can_sup = can_sup[['isbn', 'title', 'pub', 'author', 'supplier', 'currency', 'price', 'discount', 'shipping_cost','sea_shipping_cost' , 'max_qty', 'origin']]

can_sup.head()

,isbn,title,pub,author,supplier,currency,price,discount,shipping_cost,sea_shipping_cost,max_qty,origin
47,9780060000448,MIDDLE RANGE THEORIES: APPLICATION TO NURSING ...,LWW,SANDRA J. PETERSON,WC,CAD,9.95,0.1,2,2,10000,CAN
82,9780006392828,N/A,N/A,N/A,WC,CAD,19.95,0.1,2,2,10000,CAN
83,9780006392910,N/A,N/A,N/A,WC,CAD,18.95,0.1,2,2,10000,CAN
84,9780006393146,N/A,N/A,N/A,WC,CAD,19.95,0.1,2,2,10000,CAN
85,9780006393740,N/A,N/A,N/A,WC,CAD,16.99,0.1,2,2,10000,CAN


## 5. Append SupplierMegaLis, PublisherSuppliers and WesterCampus

In [27]:
all_sup = megalist_pub.append([sup_biblio_disc, can_sup], ignore_index= True)

all_sup_biblio = pd.merge(all_sup,biblio[['isbn', 'publisher']] , how='left', on = 'isbn')

all_sup_biblio['pub'] = np.where(all_sup_biblio['pub'].isna(), all_sup_biblio['publisher'], all_sup_biblio['pub'])

all_sup = all_sup_biblio[['isbn', 'title', 'pub', 'author', 'supplier', 'currency', 'price', 'discount', 'shipping_cost','sea_shipping_cost' , 'max_qty', 'origin']]
all_sup['sea_shipping_cost'] = np.where( (all_sup['origin'] == 'US') | (all_sup['origin'] == 'CAN') , all_sup['shipping_cost'], all_sup['sea_shipping_cost'])
all_sup.isna().sum()

isbn                   0
title                458
pub                  458
author               458
supplier               0
currency               0
price                  0
discount               0
shipping_cost          0
sea_shipping_cost      0
max_qty                0
origin                 0
dtype: int64

## 6. Add Fees

In [28]:
#Invoice Fee
all_sup['invoice_fee'] = np.where(     # IF
    all_sup['supplier'].str.contains('ALEK'), 0.005,   #Condition and True
    np.where(  # IF False
        (all_sup['supplier'].str.contains('LAURENTIU')) | (all_sup['supplier'].str.contains('SENAD')),0.02,   #Condition and True
        np.where(  #IF FALSE
            (all_sup['supplier'].str.contains('ARMANDO')) | (all_sup['supplier'].str.contains('FELIPE')),0.01, #Condition and True
            np.where(   #IF FALSE
                (all_sup['supplier'] == 'COINFO'), -1/11, #Condition and True
                0)   # FALSE
                )
                )
                )
# lp_fee

all_sup['lp_fee'] = np.where(
    (all_sup['supplier'].str.contains('BILLSON') & (all_sup['discount'] == 0 )), 0.03,0)

all_sup.head()

,isbn,title,pub,author,supplier,currency,price,discount,shipping_cost,sea_shipping_cost,max_qty,origin,invoice_fee,lp_fee
0,9780262530873,N/A,UNIVERSITY PRESS,N/A,BILLSON,GBP,4.99,0.38,3.47,1.0,1000,UK,0.0,0.0
1,9780262530989,N/A,UNIVERSITY PRESS,N/A,BILLSON,GBP,25.00,0.38,3.47,1.0,1000,UK,0.0,0.0
2,9781861345806,N/A,THAMES & HUDSON,N/A,ZOOKAL,AUD,28.00,0,3.47,1.0,150,AUS,0.0,0.0
3,9780262531092,N/A,UNIVERSITY PRESS,N/A,BILLSON,GBP,37.95,0.38,3.47,1.0,1000,UK,0.0,0.0
4,9781861345837,N/A,THAMES & HUDSON,N/A,ZOOKAL,AUD,35.00,0,3.47,1.0,150,AUS,0.0,0.0


## 7. import and Merge exchage rate with all suppliers

In [29]:
exch_df = pd.read_sql(exchange_rate,proc_db)
exch_df.rename(columns={'name': 'currency'}, inplace = True)
all_sup_exc = pd.merge(all_sup, exch_df, how='left', on = 'currency')
all_sup_exc.head()

,isbn,title,pub,author,supplier,currency,price,discount,shipping_cost,sea_shipping_cost,max_qty,origin,invoice_fee,lp_fee,rate
0,9780262530873,N/A,UNIVERSITY PRESS,N/A,BILLSON,GBP,4.99,0.38,3.47,1.0,1000,UK,0.0,0.0,1.241462
1,9780262530989,N/A,UNIVERSITY PRESS,N/A,BILLSON,GBP,25.00,0.38,3.47,1.0,1000,UK,0.0,0.0,1.241462
2,9781861345806,N/A,THAMES & HUDSON,N/A,ZOOKAL,AUD,28.00,0,3.47,1.0,150,AUS,0.0,0.0,0.687902
3,9780262531092,N/A,UNIVERSITY PRESS,N/A,BILLSON,GBP,37.95,0.38,3.47,1.0,1000,UK,0.0,0.0,1.241462
4,9781861345837,N/A,THAMES & HUDSON,N/A,ZOOKAL,AUD,35.00,0,3.47,1.0,150,AUS,0.0,0.0,0.687902


## 8. Cost Before exchange rate and landed cost

In [30]:
all_sup_exc['inv_price_before_exc'] = all_sup_exc['price']*(1-all_sup_exc['discount']+all_sup_exc['lp_fee']*(1+all_sup_exc['invoice_fee']))
all_sup_exc['regular_landed_cost'] = all_sup_exc['inv_price_before_exc'] * all_sup_exc['rate'] + all_sup_exc['shipping_cost']
all_sup_exc['sea_landed_cost'] = all_sup_exc['inv_price_before_exc'] * all_sup_exc['rate'] + all_sup_exc['sea_shipping_cost']
all_sup_exc.head()

,isbn,title,pub,author,supplier,currency,price,discount,shipping_cost,sea_shipping_cost,max_qty,origin,invoice_fee,lp_fee,rate,inv_price_before_exc,regular_landed_cost,sea_landed_cost
0,9780262530873,N/A,UNIVERSITY PRESS,N/A,BILLSON,GBP,4.99,0.38,3.47,1.0,1000,UK,0.0,0.0,1.241462,3.0938,7.31083,4.84083
1,9780262530989,N/A,UNIVERSITY PRESS,N/A,BILLSON,GBP,25.00,0.38,3.47,1.0,1000,UK,0.0,0.0,1.241462,15.5,22.7127,20.2427
2,9781861345806,N/A,THAMES & HUDSON,N/A,ZOOKAL,AUD,28.00,0,3.47,1.0,150,AUS,0.0,0.0,0.687902,28,22.7313,20.2613
3,9780262531092,N/A,UNIVERSITY PRESS,N/A,BILLSON,GBP,37.95,0.38,3.47,1.0,1000,UK,0.0,0.0,1.241462,23.529,32.6804,30.2104
4,9781861345837,N/A,THAMES & HUDSON,N/A,ZOOKAL,AUD,35.00,0,3.47,1.0,150,AUS,0.0,0.0,0.687902,35,27.5466,25.0766


## 9. Restrictions

In [31]:
rest = pd.read_excel("../../../../Vendors/. Restrictions\All Restricted List.xlsx")
rest.columns = map(str.lower, rest.columns)
rest.rename(columns={'isbn13': 'isbn'}, inplace = True)
rest['isbn'] = rest['isbn'].astype(str)
for col in ['isbn', 'vendor']:
    rest[col] = rest.apply(lambda x: clean_up(x[col]), axis =1)

rest = rest[['isbn','vendor']]
rest.head()

,isbn,vendor
0,9780323567428,RISKY
1,9780500292105,RISKY
2,9781936523443,RISKY
3,9780134149530,RISKY
4,9780134205588,RISKY


In [32]:
# restricted to all suppliers
risky = rest[rest['vendor'] == 'RISKY']
all_sup_exc = all_sup_exc[~all_sup_exc['isbn'].isin(list(risky['isbn']))]

In [33]:
# restriction per supplier
all_rest = pd.merge(all_sup_exc, rest, how='left', left_on = ['isbn','supplier'], right_on = ['isbn', 'vendor'] )
all_rest[~(all_rest['vendor'].isna())].head()

,isbn,title,pub,author,supplier,currency,price,discount,shipping_cost,sea_shipping_cost,max_qty,origin,invoice_fee,lp_fee,rate,inv_price_before_exc,regular_landed_cost,sea_landed_cost,vendor
24758,9781260187014,PACKAGE: ELEMENTARY STATISTICS: A STEP BY STEP...,MCGRAW,ALLAN G. BLUMAN,RAJAN,USD,276.65,0.48,0.50,0.5,10000,US,0.000,0.0,1.000000,143.858,144.358,144.358,RAJAN
42115,9781284107913,N/A,J&B,N/A,RAJAN,USD,44.95,0.38,0.50,0.5,10000,US,0.000,0.0,1.000000,27.869,28.369,28.369,RAJAN
42185,9781284126143,N/A,J&B,N/A,RAJAN,USD,79.95,0.38,0.50,0.5,10000,US,0.000,0.0,1.000000,49.569,50.069,50.069,RAJAN
43115,9780077831066,MOSAIC LEVEL 2 LISTENING/SPEAKING STUDENT BOOK...,MCGRAW,JAMI HANREDDY; ELIZABETH WHALLEY,ALEK,GBP,42.65,0.5,3.47,1.0,10000,UK,0.005,0.0,1.241462,21.325,29.9442,27.4742,ALEK
59675,9781285769431,MILADY STANDARD COSMETOLOGY,CENGAGE,MILADY,SOHRAB,SGD,83.50,0,10.00,1.2,10000,SINGAPORE,0.000,0.0,0.716869,83.5,69.8586,61.0586,SOHRAB


## 10. Getting all posible orders for cost_analysis

In [34]:
all_rest.drop(
    all_rest[
        (all_rest['supplier'] == all_rest['vendor'])
        ].index, inplace = True
)
all_rest.drop(columns=['vendor'], inplace = True)
all_rest = all_rest.sort_values(by=['regular_landed_cost'])
all_rest.head()

,isbn,title,pub,author,supplier,currency,price,discount,shipping_cost,sea_shipping_cost,max_qty,origin,invoice_fee,lp_fee,rate,inv_price_before_exc,regular_landed_cost,sea_landed_cost
2927709,9780786820771,N/A,HACHETTE LIVE,N/A,ALEK,USD,0.01,0.59,0.5,0.5,1000,US,0.005,0.0,1.000000,0.0041,0.5041,0.5041
2344345,9780194705493,N/A,OXFORD UP ELT,N/A,ALEK,GBP,0.14,0.43,0.5,0.5,10000,US,0.005,0.0,1.241462,0.0798,0.599069,0.599069
3669203,9780078022159,DATABASE SYSTEM CONCEPTS,MCGRAW,"KORTH, HENRY F., SUDARSHAN, S., SILBERSCHATZ, ...",RAJAN,USD,0.25,0.48,0.5,0.5,10000,US,0.000,0.0,1.000000,0.13,0.63,0.63
2460321,9780593115831,N/A,PENGUINYOUNG READERS,N/A,ALEK,USD,1.11,0.66,0.5,0.5,100,US,0.005,0.0,1.000000,0.3774,0.8774,0.8774
2849001,9780525579373,N/A,RH CHILDRENS BOOKS,N/A,ALEK,USD,1.11,0.66,0.5,0.5,100,US,0.005,0.0,1.000000,0.3774,0.8774,0.8774


In [35]:
all_rest = all_rest.fillna('N/A')
all_rest.head()

,isbn,title,pub,author,supplier,currency,price,discount,shipping_cost,sea_shipping_cost,max_qty,origin,invoice_fee,lp_fee,rate,inv_price_before_exc,regular_landed_cost,sea_landed_cost
2927709,9780786820771,N/A,HACHETTE LIVE,N/A,ALEK,USD,0.01,0.59,0.5,0.5,1000,US,0.005,0.0,1.000000,0.0041,0.504100,0.504100
2344345,9780194705493,N/A,OXFORD UP ELT,N/A,ALEK,GBP,0.14,0.43,0.5,0.5,10000,US,0.005,0.0,1.241462,0.0798,0.599069,0.599069
3669203,9780078022159,DATABASE SYSTEM CONCEPTS,MCGRAW,"KORTH, HENRY F., SUDARSHAN, S., SILBERSCHATZ, ...",RAJAN,USD,0.25,0.48,0.5,0.5,10000,US,0.000,0.0,1.000000,0.1300,0.630000,0.630000
2460321,9780593115831,N/A,PENGUINYOUNG READERS,N/A,ALEK,USD,1.11,0.66,0.5,0.5,100,US,0.005,0.0,1.000000,0.3774,0.877400,0.877400
2849001,9780525579373,N/A,RH CHILDRENS BOOKS,N/A,ALEK,USD,1.11,0.66,0.5,0.5,100,US,0.005,0.0,1.000000,0.3774,0.877400,0.877400


## Extra: Getting suppliers from UK, CAN, US

In [36]:
first_filter = all_rest[
    (all_rest['origin'] == 'UK')
    | (all_rest['origin'] == 'US')
    | (all_rest['origin'] == 'CAN')
    ]

first_filter.head()

,isbn,title,pub,author,supplier,currency,price,discount,shipping_cost,sea_shipping_cost,max_qty,origin,invoice_fee,lp_fee,rate,inv_price_before_exc,regular_landed_cost,sea_landed_cost
2927709,9780786820771,N/A,HACHETTE LIVE,N/A,ALEK,USD,0.01,0.59,0.5,0.5,1000,US,0.005,0.0,1.000000,0.0041,0.504100,0.504100
2344345,9780194705493,N/A,OXFORD UP ELT,N/A,ALEK,GBP,0.14,0.43,0.5,0.5,10000,US,0.005,0.0,1.241462,0.0798,0.599069,0.599069
3669203,9780078022159,DATABASE SYSTEM CONCEPTS,MCGRAW,"KORTH, HENRY F., SUDARSHAN, S., SILBERSCHATZ, ...",RAJAN,USD,0.25,0.48,0.5,0.5,10000,US,0.000,0.0,1.000000,0.1300,0.630000,0.630000
2460321,9780593115831,N/A,PENGUINYOUNG READERS,N/A,ALEK,USD,1.11,0.66,0.5,0.5,100,US,0.005,0.0,1.000000,0.3774,0.877400,0.877400
2849001,9780525579373,N/A,RH CHILDRENS BOOKS,N/A,ALEK,USD,1.11,0.66,0.5,0.5,100,US,0.005,0.0,1.000000,0.3774,0.877400,0.877400


In [37]:
isbn_list = ['9780134610993',
'9780803677074',
'9781284178418',
'9780803669260',
'9780134752556',
'9780323639088',
'9781492571186',
'9781138683143',
'9780789760500',
'9781496396273',
'9780803677111',
'9781544355658',
'9780803661134',
'9781544331430',
'9780803676787',
'9781544325040',
'9781138088870',
'9780323611077',
'9781506379616',
'9780803675797',
'9780803679917',
'9780803690141',
'9780803674882',
'9780323597791',
'9780803676459',
'9781719640022',
'9780803694736',
'9781319216801',
'9781319216801',
'9781319184568',
'9780134641690',
'9780133953145',
'9780134475585',
'9781118334324',
'9781285740621',
'9781285741550',
'9781118875865',
'9780134169040',
'9781305965720',
'9781111344191',
'9780393640342',
'9780134112107',
'9781319059811',
'9780132273589',
'9780134113593',
'9780077862510',
'9780321962584',
'9781133591887',
'9780323312073',
'9780073384429',
'9781133586845',
'9780321949912',
'9780803658783',
'9780763763381',
'9780534602710',
'9780073385372',
'9780763756093',
'9781584265306',
'9781584265320',
'9781584265290',
'9780199591152',
'9781416406914',
'9780323479820',
'9780078026881',
'9780323402101',
'9780803661158',
'9781138819078',
'9781259677588',
'9781260494198',
'9781138229778',
'9780073513942',
'9781405184588',
'9780849342707',
'9780060786830',
'9780316113502',
'9780465026425',
'9780465052714',
'9780465026425',
'9780465062980',
'9780446404662',
'9780812972764',
'9781599637594',
'9780688184742',
'9780306810121'
]

In [38]:
uk_us_can = first_filter[first_filter['isbn'].isin(isbn_list)]
uk_us_can.to_csv('retail/uk_us_can.csv', index= False)
# all_suppliers = all_rest[all_rest['isbn'].isin(isbn_list)]
# all_suppliers.to_csv('retail/all_suppliers.csv', index= False)
# elsevier = all_rest[all_rest['pub'].str.contains('ELSEVIER')]
# elsevier.to_csv('retail/elsevier.csv', index= False)
elsevier_cengage = all_rest[all_rest['pub'].str.contains('CENG') | all_rest['pub'].str.contains('ELSE')]
# elsevier_cengage.drop_duplicates(subset ="isbn", keep = False, inplace = True) 
elsevier_cengage.to_csv('retail/elsevier_cengage.csv', index= False)


In [39]:
# all_rest.to_csv('important_files/python/ca_results.csv', index= False)


